In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Load the dataset
data = pd.read_csv('creditcard.csv')

# Split the dataset into features (X) and target (y)
X = data.drop(columns=['Class']).values
y = data['Class'].values

# Standardize the dataset
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Further split the training set into three parts for the local models
X1, X_temp, y1, y_temp = train_test_split(X_train, y_train, test_size=0.67, random_state=42)
X2, X3, y2, y3 = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Function to create a model
def create_keras_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')  # For binary classification
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train local models
model1 = create_keras_model()
model1.fit(X1, y1, epochs=5, batch_size=32, verbose=0)

model2 = create_keras_model()
model2.fit(X2, y2, epochs=5, batch_size=32, verbose=0)

model3 = create_keras_model()
model3.fit(X3, y3, epochs=5, batch_size=32, verbose=0)

# Evaluate local models
loss1, accuracy1 = model1.evaluate(X_val, y_val, verbose=0)
loss2, accuracy2 = model2.evaluate(X_val, y_val, verbose=0)
loss3, accuracy3 = model3.evaluate(X_val, y_val, verbose=0)

print(f"Local Model 1 Accuracy: {accuracy1}")
print(f"Local Model 2 Accuracy: {accuracy2}")
print(f"Local Model 3 Accuracy: {accuracy3}")

# Aggregate models (simple averaging of weights)
global_model = create_keras_model()

w1 = model1.get_weights()
w2 = model2.get_weights()
w3 = model3.get_weights()

# Check if the weights have the same shape before averaging
if all(w1[i].shape == w2[i].shape == w3[i].shape for i in range(len(w1))):
    new_weights = [(w1[i] + w2[i] + w3[i]) / 3 for i in range(len(w1))]
    global_model.set_weights(new_weights)
else:
    raise ValueError("Weight shapes do not match for aggregation.")

# Evaluate global model
loss_global, accuracy_global = global_model.evaluate(X_val, y_val, verbose=0)
print(f"Global Model Accuracy: {accuracy_global}")

# Print w-value of the first layer for demonstration
print("W-value of the first layer of the global model:")
print(global_model.layers[0].get_weights()[0])

Local Model 1 Accuracy: 0.9987874627113342
Local Model 2 Accuracy: 0.9989814758300781
Local Model 3 Accuracy: 0.9991269707679749
Global Model Accuracy: 0.9973809123039246
W-value of the first layer of the global model:
[[ 0.0626101  -0.2226212   0.18559451  0.4378735  -0.14489853  0.00787769
  -0.10774463  0.4610295  -0.28234604  0.17161393]
 [ 0.15474118  0.35942972 -0.06758074 -0.02086192  0.10812905 -0.00233585
   0.00127262  0.09892038 -0.27752283 -0.22416937]
 [-0.13189137 -0.07294794  0.10221067  0.21954006  0.09373886 -0.27415505
  -0.22932374 -0.27236542  0.11389674  0.04006524]
 [ 0.12655981  0.00611955  0.18010582  0.19791444 -0.12056605  0.03839172
   0.04442382  0.0316169  -0.06482437 -0.04125282]
 [-0.09070823 -0.1547461   0.41378066  0.02309519  0.43966293 -0.05452984
  -0.08793157 -0.2195191   0.03397601 -0.38607398]
 [-0.19874464 -0.10535222  0.19925648  0.21774936  0.19204013 -0.08406327
  -0.15039515  0.19045205  0.02092324 -0.1253486 ]
 [-0.14387028 -0.16670443  0.08